In [1]:
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.feature_extraction import image
import numpy as np
import cv2


/home/abhishek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/abhishek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/abhishek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/abhishek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#Reading the input image

img = cv2.imread('2.jpg')

m,n,l = img.shape

b,g,r = cv2.split(img)

window_shape = (6, 6)

In [3]:
#sklearn library to extract all 2d patches of specified window shape

blue = extract_patches_2d(b, window_shape)
green = extract_patches_2d(g, window_shape)
red = extract_patches_2d(r, window_shape)


/home/abhishek/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/image.py:287: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  indexing_strides = arr[slices].strides


In [9]:
# Variables
Dg = []
Db = []

Bl_pixels = []
Gr_pixels = []
Re_pixels = []


In [10]:
#Estimating the pixels whose patch variation is < 0.9

for i in range(len(blue)):
    bl = blue[i]
    gr = green[i]
    re = red[i]
    
    bls = list(bl.flat)
    grls = list(gr.flat)
    rels = list(re.flat)
    
    if np.var(bl) < 0.9 or np.var(gr) < 0.9 or np.var(re) < 0.9:
        Db.extend([a - b for a, b in zip(bls, rels)])
        Dg.extend([a - b for a, b in zip(grls, rels)])

        Bl_pixels.extend(bls)
        Gr_pixels.extend(grls)
        Re_pixels.extend(rels)

In [11]:
#Getting the max of Db and Dg

maxB = Db[0]
maxG = Dg[0]
indB = 0
indG = 0

ind = 0
maxR = 0
for i in range(1,len(Db)):
    if maxB < Db[i] or maxG < Dg[i]:
        if maxR < Re_pixels[i]:
            ind = i
            maxB = Db[i]
            maxG = Dg[i]
            maxR = Re_pixels[i]

In [12]:
#Backlight Pixels
Backlight = []

Backlight.append(Bl_pixels[ind])
Backlight.append(Gr_pixels[ind])
Backlight.append(Re_pixels[ind])


In [13]:
Backlight

[150, 176, 86]

In [14]:
#reconstruct back the image from patches(Optional run)

reconsBlue = image.reconstruct_from_patches_2d(blue, (m,n))
reconsGreen = image.reconstruct_from_patches_2d(green, (m,n))
reconsRed = image.reconstruct_from_patches_2d(red, (m,n))


In [10]:
final_img = np.zeros((img.shape[0], img.shape[1], 3))

final_img[:,:,0] = reconsBlue
final_img[:,:,1] = reconsGreen
final_img[:,:,2] = reconsRed


In [11]:
cv2.imwrite('Output1.jpg',final_img)


True

In [15]:
#Marking pixels = Backlight in original image

i_ind = 0
j_ind = 0
flag = 0
for i in range(m):
    for j in range(n):
        if Backlight == list(img[i,j]):
                i_ind = i
                j_ind = j
                img[i,j] = 0
                img[i+1,j] = 0
                img[i,j+1]=0
                img[i-1,j]=0
                img[i-1,j-1]=0
                img[i+1,j+1]=0
                #flag = 1
                #break   
        #if flag == 1:
            #break

In [16]:
cv2.imwrite('Output2.jpg',img)

True

In [17]:
#Finding attenuation ratios 

Sb = (-0.00113*450 + 1.6251)
Sg = (-0.00113*540 + 1.6251)
Sr = (-0.00113*620 + 1.6251)

#Attenuation ratios

lamda_b = (Sb*Backlight[2])/(Sr*Backlight[0])

lamda_g = (Sg*Backlight[2])/(Sr*Backlight[1])

print('lamda_b : ',lamda_b, ' lamda_g : ',lamda_g)

lamda_b :  0.6924651162790698  lamda_g :  0.536416490486258


In [18]:
#Finding Initial Transmission of Red Channel

tr = []
for i in range(m):
    trans = []
    for j in range(n):
        trans.append((r[i][j] - Backlight[2])/np.max([(r[i][j] - Backlight[k]) for k in range(len(Backlight))]))
    tr.append(trans)

print(tr)

/home/abhishek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in ubyte_scalars
  import sys
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
#Refining Transmission using WLS Filter

#Matlab code refered from : http://cs.huji.ac.il/~danix/epd/wlsFilter.m
r_trans = np.array(tr)
print(r_trans)

L = np.log(r_trans + np.spacing(1))

alpha = 1.2

lamda = 0.05

smallNum = 0.0001

r,c = r_trans.shape

k = r*c

dy = np.diff(L,0,0)

old_dy = dy
absDy = abs(dy)

power = (np.power(absDy,alpha) + smallNum)

dy = np.divide(lamda,power)

dy = np.pad(dy,(1,0),'constant')

dy = dy.flatten()

[[1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 ...
 [0.         1.         1.         ... 0.3140496  0.32244897 0.32520324]
 [1.         0.01030928 1.         ... 0.30833334 0.33333334 0.32244897]
 [1.         0.01030928 1.         ... 0.31120333 0.31967214 0.30833334]]


In [21]:
dx = np.diff(L,0,1)

old_dx = dx

absDx = abs(dx)

powerDx = (np.power(absDx,alpha) + smallNum)

dx = np.divide(lamda,powerDx)

dx = np.pad(dx,(0,1),'constant')

dx = dx.flatten()

In [22]:
lapMat = np.zeros(shape=(len(dx),2))

lapMat[:,0] = dx
lapMat[:,1] = dy

d = [-r,-1]

lapMat = np.transpose(lapMat)

import scipy

reslapMat = scipy.sparse.spdiags(lapMat,d,k,k)


In [23]:
reslapMat.shape

(538900, 538900)

In [24]:
e = dx

w = np.lib.pad(dx,(r,0),'constant',constant_values=(0))


In [25]:
w = w[0:len(w)-r]

In [26]:
s = dy

n = np.lib.pad(dy,(1,0),'constant',constant_values=(0))

n = n[0:(len(n) - 1)]


In [27]:
e.shape,w.shape,s.shape,n.shape,dy.shape

((540385,), (540385,), (540385,), (540385,), (540385,))

In [28]:
D = 1-(e+w+s+n);

In [29]:
A = reslapMat

temp = np.transpose(D)

A = A + np.transpose(A) + scipy.sparse.spdiags(temp, 0, k, k);



In [30]:
A

<538900x538900 sparse matrix of type '<class 'numpy.float64'>'
	with 2688998 stored elements in Compressed Sparse Row format>

In [31]:
A.shape

(538900, 538900)

In [32]:
type(A)

scipy.sparse.csr.csr_matrix

In [33]:
from scipy.sparse.linalg import spsolve

tr_flat = r_trans.flatten()

x = spsolve(A, tr_flat)

In [34]:
x.shape,x

((538900,), array([ 1.49055165e+00,  1.44503187e+00,  1.39898462e+00, ...,
        -5.74268861e-04, -4.91312045e-04, -3.42275273e-04]))

In [35]:
refined_trans = x.reshape((r,c))

In [36]:
refined_trans.shape,refined_trans

((634, 850), array([[ 1.49055165e+00,  1.44503187e+00,  1.39898462e+00, ...,
         -4.36589657e-02, -4.81708542e-02, -5.35051888e-02],
        [ 1.00000000e+00, -6.56234180e-02, -7.61341993e-02, ...,
          1.29242576e+00,  1.30152768e+00,  1.31063473e+00],
        [ 1.31984639e+00,  1.32937416e+00,  1.33608657e+00, ...,
          9.79917184e-01,  9.35320640e-01,  8.91272598e-01],
        ...,
        [ 2.92178885e-01,  2.96645119e-01,  2.98931414e-01, ...,
          4.67728164e-01,  4.66088519e-01,  4.64915461e-01],
        [ 4.67417996e-01,  4.68874933e-01,  4.67975978e-01, ...,
         -2.90797911e-02, -2.43455509e-02, -1.69339811e-02],
        [-1.70549274e-02, -1.79949566e-02, -2.60492164e-02, ...,
         -5.74268861e-04, -4.91312045e-04, -3.42275273e-04]]))

In [37]:
'''
import numpy.linalg as lin

x_out,resid_out,rank_out,s_out = np.linalg.lstsq(A,tr_flat)
'''
'''
from scipy.sparse.linalg import lsqr

#Run this part of code.

x_out,resid_out,rank_out,s_out = lsqr(A,tr_flat)[:4]
'''

'''
#x = np.linalg.solve(A, np.transpose(tr_flat))

from scipy import linalg
x = linalg.solve(A, tr_flat)
'''

'\n#x = np.linalg.solve(A, np.transpose(tr_flat))\n\nfrom scipy import linalg\nx = linalg.solve(A, tr_flat)\n'

In [30]:
#Normalizing backlight 

normBacklight = [item/256 for item in Backlight]

In [31]:
normBacklight.extend([(1-item) for item in normBacklight])

In [32]:
denoMax = max(normBacklight)

In [33]:
denoMax

0.6875

In [34]:
n

array([0.        , 0.        , 0.        , ..., 0.04309274, 0.04152513,
       0.04270091], dtype=float32)

In [35]:
#Estimating K

K_list = []

t_b,t_g,t_r = cv2.split(img)

for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        tempLS = []
        tempLS.append(t_b[i][j]/256 - normBacklight[0])
        tempLS.append(t_g[i][j]/256 - normBacklight[1])
        tempLS.append(t_r[i][j]/256 - normBacklight[2])
        K_list.append(max(tempLS)/denoMax)


In [36]:
K = max(K_list)
K

0.8636363636363636

In [37]:
normB = t_b/256
normG = t_g/256
normR = t_r/256
normImg = img/256

In [38]:
normB,normG,normR,normImg

(array([[0.390625  , 0.390625  , 0.3828125 , ..., 0.44921875, 0.453125  ,
         0.453125  ],
        [0.38671875, 0.390625  , 0.3828125 , ..., 0.44921875, 0.453125  ,
         0.453125  ],
        [0.39453125, 0.390625  , 0.390625  , ..., 0.453125  , 0.44921875,
         0.44921875],
        ...,
        [0.25390625, 0.25      , 0.2421875 , ..., 0.50390625, 0.50390625,
         0.5078125 ],
        [0.23046875, 0.25390625, 0.2421875 , ..., 0.48046875, 0.5078125 ,
         0.4921875 ],
        [0.23046875, 0.25390625, 0.2421875 , ..., 0.484375  , 0.48828125,
         0.47265625]]),
 array([[0.421875  , 0.421875  , 0.4296875 , ..., 0.5234375 , 0.52734375,
         0.52734375],
        [0.41796875, 0.42578125, 0.4375    , ..., 0.52734375, 0.53125   ,
         0.53125   ],
        [0.4375    , 0.4453125 , 0.44921875, ..., 0.5390625 , 0.53515625,
         0.53515625],
        ...,
        [0.3984375 , 0.39453125, 0.390625  , ..., 0.73046875, 0.7265625 ,
         0.73046875],
        [0.3

In [39]:
lambda_r = max(lamda_b,lamda_g)

In [40]:
lambda_r,lamda_b,lamda_g

(0.6924651162790698, 0.6924651162790698, 0.536416490486258)

In [112]:
#Obaining the scene radiance

J_b = []
J_g = []
J_r = []

for i in range(img.shape[0]):
    tB = []
    tG = []
    tR = []
    for j in range(img.shape[1]):
        tB.append((normB[i][j] - normBacklight[0])/(max((K*(refined_trans[i][j]**lamda_b)),0.1)) + normBacklight[0])
        tG.append((normG[i][j] - normBacklight[1])/(max((K*(refined_trans[i][j]**lamda_g)),0.1)) + normBacklight[1])
        tR.append((normR[i][j] - normBacklight[2])/(max((K*(refined_trans[i][j]**lambda_r)),0.1)) + normBacklight[2])
    J_b.append(tB)
    J_g.append(tG)
    J_r.append(tR)


/home/abhishek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/home/abhishek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/home/abhishek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


In [127]:
#Since majority of values were nan, I replaced them with 127 as of now.

for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        if np.isnan(J_b[i][j]):
            J_b[i][j] = 127
        else:
            J_b[i][j] = J_b[i][j]*256
        if np.isnan(J_g[i][j]):
            J_g[i][j] = 127
        else:
            J_g[i][j] = J_g[i][j]*256
        if np.isnan(J_r[i][j]):
            J_r[i][j] = 127
        else:
            J_r[i][j] = J_r[i][j]*256

In [128]:
outputImg = np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)

outputImg[:,:,0] = np.array(J_b)
outputImg[:,:,1] = np.array(J_g)
outputImg[:,:,2] = np.array(J_r)


In [129]:
outputImg.shape

(634, 850, 3)

In [130]:
cv2.imwrite('Try1Output1.png',outputImg)

True